# Initial Discovery

## Context

I created this notebook, because I realized that in my initial `test.py` script I was documenting my code in a way that was more suitable to a python notebook. With that being siad, I will be rearranging some of the code to better suit this more narrative style. Hopefully this context will be helpful for anyone who was wondering.

With that out of the way, let's dive in.

## Import required libraries

In [58]:
from pypdf import PdfReader as pr
import numpy as np
import re

## Read in the data

In this case we will be working with the course catalog found at this [link]() for the 2025 Fall Term courses offered at the City College of San Francisco (CCSF).

In [3]:
# Create an instance of the `PdfReader` class
reader = pr("pdfs\ccsf_fall-2025-credit-classes.pdf")

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\chris\AppData\Local\Temp\ipykernel_33508\3247992145.py:2: SyntaxWarning: invalid escape sequence '\c'
  reader = pr("pdfs\ccsf_fall-2025-credit-classes.pdf")


## Examine the first page

In [4]:
first_page = reader.pages[0].extract_text().split('\n')

### "Meta" data

I figured it would be nice to get some metadata about the courses.

In [5]:
# Contains the information about the context of the courses in this file
meta = first_page[0].split()
print(meta)

['CREDIT', 'FALL', '2025CCSF', 'SCHEDULE', 'OF', 'CLASSES']


In [6]:
# Weird space between hard coding and kind of not?
meta = {
    # Handling the combined year and college name
    'school': meta[2][4:],
    'are_credit_courses': True if meta[0] == 'CREDIT' else False,
    'term': meta[1],
    # Handling the combined year and college name
    'year': meta[2][:4] 
}

meta

{'school': 'CCSF', 'are_credit_courses': True, 'term': 'FALL', 'year': '2025'}

### Column Headers

In this document, which you can view [here](), you can see that there are several columns used to organize information pertaining to each course. Let's check out what the `reader` object is returning.

In [7]:
first_page[1]

'CRN  SEC  TYPE  D AYS TIMES  D ATES L OCATION  C AMPUS  INSTR UCTOR'

Because several of the column titles have odd spacing, I felt that it was easier to just hardcode the column titles:

In [8]:
column_headers = [
    'CRN',
    'SEC',
    'TYPE',
    'DAYS',
    'TIMES',
    'DATES',
    'LOCATION',
    'CAMPUS',
    'INSTRUCTOR'
]

### Parsing the Data

It became clear quite quickly that the returned text data would not be very clear. So I needed to find a way to parse the data but also retain its hierarchy.

#### Troubleshooting

We have `.extract_text()`, which we used for the work above, but that didn't provide much insight on hierarchy. Printing out the text below, we can see that the footer (e.g., "REGISTER ONLINE TODAY!") is returned before any of the actual department or course information.

Additionally, we still get the weird spacing between some of the column titles and the actual course information pertaining to each column (e.g., "L ec" for the "TYPE" column, or "T R" for "DAYS", etc.).

In [9]:
# Feel free to change how much you want printed out
first_page[:10]

['CREDIT FALL 2025CCSF SCHEDULE OF CLASSES',
 'CRN  SEC  TYPE  D AYS TIMES  D ATES L OCATION  C AMPUS  INSTR UCTOR',
 'REGISTER ONLINE TODAY!  1',
 'LAST UPDATED: 6/27/2025, 4:30PM',
 'Academic Achievement & Personal Success',
 'AAPS 103: Orientation to College Transfer 3 .0',
 'PREREQ: Completion of or concurrent enrollment in: ENGL C1000.  ',
 '70482 0 08  L ec  T R  0 9:40-10:55AM  0 8/19-12/19  M IC 254  M ission  R ivera',
 ' ',
 'Accounting']

With some perusing, you can find that there are different "extraction modes" (`plain` vs `layout`). `plain` is the default and legacy option, so let's try `layout`.

In [14]:
# Converting to a list so we can print in a for loop
first_page = reader.pages[0].extract_text(extraction_mode='layout').split('\n')

lines = 10
for line in range(lines):
    print(first_page[line])

 Academic Achievement & Personal Success
 AAPS 103: Orientation to College Transfer                                                                                                                   3.0
          PREREQ: Completion of or concurrent enrollment in: ENGL C1000.
     70482      008       Lec        TR         09:40 -10:55AM          08/19 -12/19     MIC 254                Mission           Rivera



 Accounting
 ACC T 1: Financial Accounting                                                                                                                               5.0
          Recommended Prep: (Readiness for college -level English or ESL 188) and BSMA 68.


Something important to note is that this approach seems to drop the header information (up to the 10th line) but does keep the overall layout of the document well preserved **and** it does not create splits in the course information like the `plain` method did.

*However*, there is still the issue of understanding the hierarchy. We could use regular expressions and conditionals to try and map out a predictable pattern for the text, but that *feels* inefficient and the documentation also makes reference that it is "very hard to guarantee correct whitespaces."

So I looked into this method some more and found that you can use a 'visitor function' which, for the purposes of this project, can provide more information about the text being read by the extractor. This does present another issue, as 'visitor functions' cannot be used with the `layout` "extraction mode".

Nevertheless, let's inspect this new approach:

###### Method Arguments

If you would like to know more about this please follow this [link](https://pypdf.readthedocs.io/en/stable/user/extract-text.html). But I will paste their explanation of the visitor function's arguments below:

- text: the current text (as long as possible, can be up to a full line)
- user_matrix: current matrix to move from user coordinate space (also known as CTM)
- tm_matrix: current matrix from text coordinate space
- font_dictionary: full font dictionary
- font_size: the size (in text coordinate space)

###### *Uncollapse the above section if you'd like an explanation of the method arguments

The documentation shows that we pass the visitor function through our `extract_text` method, which allows us to obtain some extra information about the text that is parsed by the extractor - namely `font_size`. 

Given how the document is structured, it could be useful to use the `font_size` to distinguish between certain sections on each page. To obtain this, we need to accumulate the font sizes identfied in the extractor.

In [18]:
# Define an accumulator to collect the font sizes and the actual text processed
font_sizes = []
processed_text = []

# Define a visitor function
def visitor_func(text, cm, tm, font_dict, font_size):
    # Since we don't know how many items will be processed we will have to append new items
    font_sizes.append(font_size)
    processed_text.append(text)

# Extract/process the text on the first page
first_page = reader.pages[0].extract_text(visitor_text=visitor_func)

# Debug the information gathered
for i, (font_size, text) in enumerate(zip(font_sizes, processed_text)):
    if i >= 10:
        break
    print(f"Item {i}: ({font_size}) - {text}")

Item 0: (12.0) - 
Item 1: (12.0) - 
Item 2: (1.0) - CREDIT FALL 2025CCSF SCHEDULE OF CLASSES
Item 3: (1.0) - 
Item 4: (1.0) - 

Item 5: (1.0) - CRN
Item 6: (1.0) - 
Item 7: (1.0) -   
Item 8: (1.0) - 
Item 9: (1.0) - SEC


So, the result was pretty confusing. How is it that the font size for the largest text on the document (Item 2) has a font size of 1? I know that we already manually parsed the header, but if you increase the number of lines the previous cell prints, you will see similarly concerning/unexpected outputs.

Fortunately, there are other arguments that I passed over that can still be useful. The argument `tm` is defined as the "current matrix from text coordinate space", and the documentation later goes on to say the following:

"*The matrix stores six parameters. The first four provide the rotation/scaling matrix and the last two provide the translation (horizontal/vertical). It is recommended to use the user_matrix as it takes into account all transformations.*"

Let's take a look at what the `tm` argument returns:

In [ ]:
# Define an accumulator to collect the font sizes and the actual text processed
font_sizes = []
processed_text = []
tms = []

# Define a visitor function
def visitor_func(text, cm, tm, font_dict, font_size):
    # Since we don't know how many items will be processed we will have to append new items
    font_sizes.append(font_size)
    processed_text.append(text)
    # * Added in the tm accumulator
    tms.append(tm)

# Extract/process the text on the first page
first_page = reader.pages[0].extract_text(visitor_text=visitor_func)

# Debug the information gathered
for i, (font_size, tm, text) in enumerate(zip(font_sizes, tms, processed_text)):
    if i >= 10:
        break
    print(f"Item {i}: ({font_size}) - {tm} -{text}")

Item 0: (12.0) - [1.0, 0.0, 0.0, 1.0, 0.0, 0.0] -
Item 1: (12.0) - [1.0, 0.0, 0.0, 1.0, 0.0, 0.0] -
Item 2: (1.0) - [16.0, 0.0, 0.0, 16.0, 36.0798, 750.1841] -CREDIT FALL 2025CCSF SCHEDULE OF CLASSES
Item 3: (1.0) - [18.0, 0.0, 0.0, 18.0, 179.0648, 749.4571] -
Item 4: (1.0) - [1.0, 0.0, 0.0, 1.0, 0.0, 0.0] -

Item 5: (1.0) - [9.0, 0.0, 0.0, 9.0, 55.19, 732.15] -CRN
Item 6: (1.0) - [9.0, 0.0, 0.0, 9.0, 55.19, 732.15] -
Item 7: (1.0) - [1.0, 0.0, 0.0, 1.0, 0.0, 0.0] -  
Item 8: (1.0) - [9.0, 0.0, 0.0, 9.0, 72.677, 732.15] -
Item 9: (1.0) - [9.0, 0.0, 0.0, 9.0, 92.306, 732.15] -SEC


Given that the first four elements are said to pertain to rotation and scaling, and only the 1st and 4th of those elements are filled (you can print out more to confirm), I will opt to see if the 4th element would be a suitable marker for font size:

In [22]:
# Define an accumulator to collect the font sizes and the actual text processed
font_sizes = []
processed_text = []

# Define a visitor function
def visitor_func(text, cm, tm, font_dict, font_size):
    # Since we don't know how many items will be processed we will have to append new items
    font_sizes.append(tm[3]) # I'm now using the 4th element of tm for the font size
    processed_text.append(text)

# Extract/process the text on the first page
first_page = reader.pages[0].extract_text(visitor_text=visitor_func)

# Debug the information gathered
for i, (font_size, text) in enumerate(zip(font_sizes, processed_text)):
    if i >= 10:
        break
    print(f"Item {i}: ({font_size}) - {text}")

Item 0: (1.0) - 
Item 1: (1.0) - 
Item 2: (16.0) - CREDIT FALL 2025CCSF SCHEDULE OF CLASSES
Item 3: (18.0) - 
Item 4: (1.0) - 

Item 5: (9.0) - CRN
Item 6: (9.0) - 
Item 7: (1.0) -   
Item 8: (9.0) - 
Item 9: (9.0) - SEC


This is very promising!

I'm interested to see the distribution and examples of the font sizes and their corresponding text look like across the first page (as a sample of the entire document).

Here's how I did this:

In [25]:
# Since we will be looping, I want to store the page information to be more efficient
first_page = reader.pages[0]

# This is useful for the loop below, but needs to be created before the method
font_limit = 0
# Since the visitor function can collect lines of text
lines = []        
# This will store the information for each font size throughout the first page
_dict = {}

def visitor_func(text, cm, tm, font_dict, font_size):
    # I just want to collect font and text information for a specific size one at a time
    if tm[3] == font_limit: 
        # I don't want to sift through clutter when looking at printed examples
        if text not in  ('', ' ', "'  '", '\n'):
            # I use repr just to get more insight into the characters included in each string
            lines.append(repr(text))

# Arbitrarily chose 20 because I saw that most examples were below 18
for i in range(20):
    # The index controls the font size
    font_limit = i + 1
    
    # Do the extraction
    first_page.extract_text(visitor_text=visitor_func, extraction_mode="plain")
    # _list = lines.copy()
    _dict[font_limit] = (len(lines), lines)
    lines = []
    
for k, v in _dict.items():
    if v[0] > 0:
        print(k, v)

1 (10, ["'  '", "'  '", "'  '", "'  '", "'  '", "'  '", "'CREDIT FALL 2025CCSF SCHEDULE OF CLASSES\\nCRN  SEC  TYPE  D AYS TIMES  D ATES L OCATION  C AMPUS  INSTR UCTOR'", "'  '", "'  '", "'  '"])
9 (175, ["'CRN'", "'SEC'", "'TYPE'", "'D AYS'", "'TIMES'", "'D ATES'", "'L OCATION'", "'C AMPUS'", "'INSTR UCTOR'", "'PREREQ: Completion of or concurrent enrollment in: ENGL C1000.  '", "'70482'", "'0 08'", "'  '", "'L ec'", "'  '", "'T R'", "'  '", "'0 9:40-10:55AM'", "'  '", "'0 8/19-12/19'", "'  '", "'M IC 254'", "'  '", "'M ission'", "'  '", "'R ivera'", "'Recommended Prep: (Readiness for college-level English or ESL 188) and BSMA 68.  '", "'70243'", "'0 01'", "'  '", "'L ec'", "'  '", "'M TWRF'", "'  '", "'1 0:10-11:00AM'", "'  '", "'0 8/18-12/19'", "'  '", "'C LOU 229'", "'  '", "'O cean'", "'  '", "'Y run'", "'70244'", "'0 02'", "'  '", "'L ec'", "'  '", "'M TWRF'", "'  '", "'1 1:10-12:00PM'", "'  '", "'0 8/18-12/19'", "'  '", "'C LOU 229'", "'  '", "'O cean'", "'  '", "'Y run'", "'702

Eureka! This output tells me the following:
 - Font 9: Paragraph text, specific course information (CRN, SEC, etc.)
 - Font 11: Footer data
 - Font 14: Department, course title, number of units
 - Font 16: Main title of the document

So, we still see that there is weird separation of words, but I can do either of the following:

1. I can just remove the whitespace for each of the words because they were correctly included as one string.
2. I can ignore it and just use this approach to identify departments, course titles, units, and the overal page title, **and** I can use the `layout` extraction mode separately for the other text.

At this point, I'm opting for option 2.

Before I do that, however, I want to extend this inspection to the entire document. Since this is likely to be a slow process I will define a variable that will control if the operation will execute and will be set to `False` after the information is printed.

In [ ]:
# Apologies for the long name, but I wanted to make it descriptive
execute_document_font_distribution_extraction = False

# Throwing error to ensure the next cell isn't run if I forget and do run all
raise Exception("Do you really want to re-run the next cell?")

In [ ]:
# We don't need this to run all the time, so I added a safeguard
if execute_document_font_distribution_extraction:
    # Importing a module to do some cleaner printing
    from IPython.display import clear_output
    
    # Importing default_dict to make accumulation easier
    from collections import defaultdict
    
    # Doing this here instead of every loop
    num_pages = len(reader.pages)
         
    # This will store the information for each font size throughout the first page
    document_font_size_dict = defaultdict(lambda: [0, []]) # Needs to be outside of the loop to not erase previous page data
    
    # Iterate through each page and extract and process text/font size
    for page_number, page in enumerate(reader.pages):
        # This is useful for the loop below, but needs to be created before the method
        font_limit = 0
        # Since the visitor function can collect lines of text
        lines = []

        def visitor_func(text, cm, tm, fontdocument_font_size_dict, font_size):
            # I just want to collect font and text information for a specific size one at a time
            if tm[3] == font_limit: 
                # I don't want to sift through clutter when looking at printed examples
                if text not in  ('', ' ', "'  '", '\n'):
                    # I use repr just to get more insight into the characters included in each string
                    lines.append(repr(text))

        # Arbitrarily chose 20 because I saw that most examples were below 18
        for i in range(20):
            # The index controls the font size
            font_limit = i + 1
            
            # Do the extraction
            page.extract_text(visitor_text=visitor_func, extraction_mode="plain")
            
            # Assign the information
            document_font_size_dict[font_limit][0] += len(lines)
            # This let's us see a breakdown of what was extracted on each page
            document_font_size_dict[font_limit][1].append(lines)
            
            # Reset this accumulator to not duplicate information across key-value pairs
            lines = []
            
        # Print an update
        clear_output(wait=True)
        print(f"Completed page {page_number + 1} out of {num_pages}")
        
    # Debug distribution collected
    for k, v in document_font_size_dict.items():
        if v[0] > 0:
            print(k, v)

Completed page 280 out of 280
1 [2592, [["'  '", "'  '", "'  '", "'  '", "'  '", "'  '", "'CREDIT FALL 2025CCSF SCHEDULE OF CLASSES\\nCRN  SEC  TYPE  D AYS TIMES  D ATES L OCATION  C AMPUS  INSTR UCTOR'", "'  '", "'  '", "'  '"], ["'  '", "'  '", "'  '", "'  '", "'  '", "'  '", "'FALL 2025 CREDITCCSF SCHEDULE OF CLASSES\\nCRN  SEC  TYPE  D AYS TIMES  D ATES L OCATION  C AMPUS  INSTR UCTOR'", "'  '"], ["'  '", "'  '", "'  '", "'  '", "'  '", "'  '", "'CREDIT FALL 2025CCSF SCHEDULE OF CLASSES\\nCRN  SEC  TYPE  D AYS TIMES  D ATES L OCATION  C AMPUS  INSTR UCTOR'", "'  '", "'  '"], ["'  '", "'  '", "'  '", "'  '", "'  '", "'  '", "'FALL 2025 CREDITCCSF SCHEDULE OF CLASSES\\nCRN  SEC  TYPE  D AYS TIMES  D ATES L OCATION  C AMPUS  INSTR UCTOR'", "'  '"], ["'  '", "'  '", "'  '", "'  '", "'  '", "'  '", "'CREDIT FALL 2025CCSF SCHEDULE OF CLASSES\\nCRN  SEC  TYPE  D AYS TIMES  D ATES L OCATION  C AMPUS  INSTR UCTOR'", "'  '", "'  '", "'  '"], ["'  '", "'  '", "'  '", "'  '", "'  '", "'  '", "

Now that we have run this for the entire document, we can be extra sure that the formatting is consistently identified within the first 20 font sizes. Let's march on.

#### Implementing Plan

My plan going forward is to use the font sizes that correlate with headers, and the `layout` extraction method for everything else. To do that, I need to be able to accurately classify these lines of text. Let's take a look at that data now.

In [60]:
# Get unique list of strings for each relevant font sizes

# * Font size = 16
text_16 = []
for text_list in document_font_size_dict[16][1]:
    text_16 += text_list

# Generate a unique set of lines
text_16 = set(text_16)
print(text_16)

# * Font size = 14
text_14 = []
for text_list in document_font_size_dict[14][1]:
    text_14 += text_list

# We don't want to generate a unique set of lines here actually
print(text_14)


{"'FALL 2025 CREDITCCSF SCHEDULE OF CLASSES'", "'CREDIT FALL 2025CCSF SCHEDULE OF CLASSES'"}
["'Academic Achievement & Personal Success'", "'AAPS 103: Orientation to College Transfer'", "'3 .0'", "'Accounting'", "'ACCT 1: Financial Accounting'", "'5 .0'", "'ACCT 2: Managerial Accounting'", "'5 .0'", "'ACCT 51: Intermediate Accounting'", "'5 .0'", "'ACCT 59: Federal Income Tax'", "'3 .0'", "'Administration of Justice'", "'ADMJ 51: Juvenile Procedures'", "'3 .0'", "'ADMJ 52: Concepts of Criminal Law'", "'3 .0'", "'ADMJ 53: Legal Aspects of Evidence'", "'3 .0'", "'ADMJ 54: Principles and Procedures of the Justice System'", "'3 .0'", "'ADMJ 57: Introduction to Administration of Justice'", "'3 .0'", "'ADMJ 62: Criminal Investigation'", "'3 .0'", "'ADMJ 64: Progressive Policing in the 21st Century'", "'3 .0'", "'ADMJ 68: Criminal Justice Report Writing'", "'3 .0'", "'ADMJ 69: Crime Scene Documentation'", "'3 .0'", "'ADMJ 70A: Patrol Procedures'", "'3 .0'", "'ADMJ 72: Police Work Experience'"

We can skip the text of font size 16 and just focus on the font size of 14. Looking at the examples it seems like it might be a safe bet to parse on strings that contain a colon (course titles vs departments), and separating out those that are numbers (units). Let's see what that yields.

In [117]:
# def department_criteria():
#     return {
#         'title': '',
#         'courses': []
#     }
def course_information(crn = '', sec = '', type = '', days = '', times = '', dates = '', location = '', campus = '', instructor = '', units = []):
    return {
        'crn': crn,
        'sec': sec,
        'type': type,
        'days': days,
        'times': times,
        'dates': dates,
        'location': location,
        'campus': campus,
        'instructor': instructor,
        'units': units
    }

# Storing the departments, their courses, and their courses' data 
departments = defaultdict(str)

current_department = ''
current_course = ''
# Min, Max
current_units = []

# Generate a dictionary --> json file?
for text in text_14:
    # If the string represents a course title
    if ':' in text:
        # Assigning to a variable will be helpful for when we assign courses to a department's course list
        current_course = text
        
        # Save the current course's information
        departments[current_department][current_course] = {}
    else:
        # If the text represents the units for the course
        if re.search('[0-9]', text):
            # Clean the data
            text = re.sub("[^0-9.-]", '', text) # Instead of removing characters that I couldn't identify, I just removed everything I didn't want
            # Cast as float in case there are courses with half units??
            text = [float(course_units) for course_units in text.split('-')]
            
            
            # Update the course criteria
            departments[current_department][current_course] = course_information(units=text)    
        # Department title
        else:
            # Similar to above, assigning course title to a variable will be helpful for units
            current_department = text
            
            # #  Add a new department to the dictionary
            departments[current_department] = {}

print(departments)

defaultdict(<class 'str'>, {"'Academic Achievement & Personal Success'": {"'AAPS 103: Orientation to College Transfer'": {'crn': '', 'sec': '', 'type': '', 'days': '', 'times': '', 'dates': '', 'location': '', 'campus': '', 'instructor': '', 'units': [3.0]}}, "'Accounting'": {"'ACCT 1: Financial Accounting'": {'crn': '', 'sec': '', 'type': '', 'days': '', 'times': '', 'dates': '', 'location': '', 'campus': '', 'instructor': '', 'units': [5.0]}, "'ACCT 2: Managerial Accounting'": {'crn': '', 'sec': '', 'type': '', 'days': '', 'times': '', 'dates': '', 'location': '', 'campus': '', 'instructor': '', 'units': [5.0]}, "'ACCT 51: Intermediate Accounting'": {'crn': '', 'sec': '', 'type': '', 'days': '', 'times': '', 'dates': '', 'location': '', 'campus': '', 'instructor': '', 'units': [5.0]}, "'ACCT 59: Federal Income Tax'": {'crn': '', 'sec': '', 'type': '', 'days': '', 'times': '', 'dates': '', 'location': '', 'campus': '', 'instructor': '', 'units': [3.0]}}, "'Administration of Justice'":

This worked well